In [ ]:
import datetime
import pandas as pd
import gymnasium as gym
import gym_anytrading
import numpy as np
import datetime
from gym_anytrading.envs import StocksEnv
import random
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout 
from keras.optimizers import Adam
import subprocess

In [ ]:
import yfinance as yf
import pandas as pd

def get_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    return stock_data

# Set the date range for the historical data
start_date = "2001-01-01"
end_date = "2023-01-01"  # Adjust the end date as needed

# Get S&P 500 historical data
sp500_data = get_stock_data("^GSPC", start_date, end_date)

# Print the first few rows of the dataset
print(sp500_data.head())

# Save the data to a CSV file if needed
sp500_data.to_csv('sp500_data.csv', index=False)


In [ ]:
import pandas as pd
from finta import TA
from environment import CustomTradingEnv

# Assuming you have the dataset saved as 'sp500_data.csv'
df = pd.read_csv('sp500_data.csv')

# Check the length of the original dataframe
print("Length before adding indicators:", len(df))

# Add technical indicators
df['TEMA'] = TA.TEMA(df, 30)
df['ER'] = TA.ER(df)
df['RSI'] = TA.RSI(df)
df['OBV'] = TA.OBV(df)
df['STOCH'] = TA.STOCH(df)

# Fill NaN values
df.fillna(0, inplace=True)

# Print dataframe information
print(df.info())

# Print the first few rows of the dataframe
print(df.head())

# Define window_size based on the number of historical data points you want to consider
window_size = 21

# Split the dataframe into training and testing sets
train_df = df.iloc[:int(0.8 * len(df))]
test_df = df.iloc[int(0.8 * len(df)):]

# Define your frame_bound based on the training data
frame_bound_train = (window_size, len(train_df) - 1)

# Create your custom trading environment for training
train_env = CustomTradingEnv(train_df, window_size, frame_bound_train, balance=100000, render_mode=None)

# Define your frame_bound based on the testing data
frame_bound_test = (window_size, len(test_df) - 1)

# Create your custom trading environment for testing
test_env = CustomTradingEnv(test_df, window_size, frame_bound_test, balance=100000, render_mode=None)


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import torch.optim as optim
from dqn_2 import Agent, Utils
# Define Training Parameters
num_episodes = 1000
initial_epsilon = 1.0
starting_mem_len = 32  # Adjust as needed
max_mem_len = 100000  # Adjust as needed
learn_rate = 0.001

# Initialize DQN Agent
possible_actions = [0, 1]  # Adjust based on your action space
agent = Agent(possible_actions, starting_mem_len, max_mem_len, initial_epsilon, learn_rate)

agent.model.load_state_dict(torch.load("results/recent_weights_1.pt"))

# Training Loop
utils = Utils()

for episode in range(1, num_episodes + 1):
    episode_score, total_profit, _ = utils.play_episode(train_env, agent, episode, debug=False)
    print(f"Episode {episode}/{num_episodes} - Score: {episode_score}, Profit: {total_profit}")

# Save Model Weights
torch.save(agent.model.state_dict(), 'final_weights.pt')
print('\nFinal weights saved!')

# Evaluate on Test Set
test_episodes = 10  # Adjust based on your needs
test_scores = []

for episode in range(1, test_episodes + 1):
    episode_score, _ = utils.play_episode(test_env, agent, episode, debug=False)
    test_scores.append(episode_score)
    print(f"Test Episode {episode}/{test_episodes} - Score: {episode_score}")

avg_test_score = np.mean(test_scores)
print(f"\nAverage Test Score: {avg_test_score}")
